In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import numpy as np
from torch.utils.data import DataLoader, TensorDataset


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
full_X_train_transformed = pd.read_csv('CSV/pre-processed/full_X_train_transformed')
full_y_train = pd.read_csv('CSV/pre-processed/full_y_train')
full_y_train = full_y_train.drop(columns=['id'])
full_y_train = full_y_train.iloc[:, 0]

full_X_test_transformed = pd.read_csv('CSV/pre-processed/full_X_test_transformed')


In [7]:
# Extracting only the image data from the full_X_train_transformed DataFrame
image_data_train = full_X_train_transformed.iloc[:, -40000:]
image_data_test = full_X_test_transformed.iloc[:, -40000:]

In [8]:
image_data_train = image_data_train.values.reshape(-1, 200, 200)
image_data_test = image_data_test.values.reshape(-1, 200, 200)

In [9]:
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(full_y_train)

# Splitting 20% of the training data as a validation set
X_train, X_val, y_train_encoded_split, y_val_encoded_split = train_test_split(
    image_data_train, y_train_encoded, test_size=0.1, stratify=y_train_encoded, random_state=42
)

print(f"Training set size: {X_train.shape}")
print(f"Validation set size: {X_val.shape}")


Training set size: (891, 200, 200)
Validation set size: (99, 200, 200)


In [10]:
# Convert grayscale images to 3-channel (RGB) format
X_train_rgb = np.stack((X_train,) * 3, axis=-1)
X_val_rgb = np.stack((X_val,) * 3, axis=-1)

# Convert the RGB numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train_rgb.transpose(0, 3, 1, 2), dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val_rgb.transpose(0, 3, 1, 2), dtype=torch.float32).to(device)

# Normalize using ImageNet statistics
mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to(device)
std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to(device)
X_train_normalized = (X_train_tensor - mean) / std
X_val_normalized = (X_val_tensor - mean) / std


In [11]:
# from torchvision.models.resnet import ResNet50_Weights

# # The image data tensors were already created as X_train_normalized and X_val_normalized
# # Convert label data to tensors
# y_train_tensor = torch.tensor(y_train_encoded_split, dtype=torch.int64).to(device)
# y_val_tensor = torch.tensor(y_val_encoded_split, dtype=torch.int64).to(device)

# # Define the model using ResNet50
# n_classes = full_y_train.nunique()
# model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
# model.fc = nn.Linear(model.fc.in_features, n_classes)  # Modify the final layer

# # Optionally, freeze all layers
# for param in model.parameters():
#     param.requires_grad = False
# # Unfreeze the final layer
# for param in model.fc.parameters():
#     param.requires_grad = True

# model.to(device)
# print(model)

# # Define the loss and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [12]:
# from torchvision.models.resnet import ResNet101_Weights

# # The image data tensors were already created as X_train_normalized and X_val_normalized
# # Convert label data to tensors
# y_train_tensor = torch.tensor(y_train_encoded_split, dtype=torch.int64).to(device)
# y_val_tensor = torch.tensor(y_val_encoded_split, dtype=torch.int64).to(device)

# # Define the model using ResNet101
# n_classes = full_y_train.nunique()
# model = models.resnet101(weights=ResNet101_Weights.IMAGENET1K_V1)  # Load the ResNet101 with ImageNet weights
# model.fc = nn.Linear(model.fc.in_features, n_classes)  # Modify the final layer to match your number of classes

# # Optionally, freeze all layers
# for param in model.parameters():
#     param.requires_grad = False
# # Unfreeze the final layer
# for param in model.fc.parameters():
#     param.requires_grad = True

# model.to(device)
# print(model)

# # Define the loss and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [13]:
from torchvision.models.resnet import ResNet152_Weights

# The image data tensors were already created as X_train_normalized and X_val_normalized
# Convert label data to tensors
y_train_tensor = torch.tensor(y_train_encoded_split, dtype=torch.int64).to(device)
y_val_tensor = torch.tensor(y_val_encoded_split, dtype=torch.int64).to(device)

# Define the model using ResNet152
n_classes = full_y_train.nunique()
model = models.resnet152(weights=ResNet152_Weights.IMAGENET1K_V1)  # Load the ResNet152 with ImageNet weights
model.fc = nn.Linear(model.fc.in_features, n_classes)  # Modify the final layer to match your number of classes

# Optionally, freeze all layers
for param in model.parameters():
    param.requires_grad = False
# Unfreeze the final layer
for param in model.fc.parameters():
    param.requires_grad = True

model.to(device)
print(model)

# Define the loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
# Training parameters
epochs = 100
batch_size = 32
patience = 10  # Number of epochs to wait for improvement before stopping

# Create datasets
train_dataset = TensorDataset(X_train_normalized, y_train_tensor)
val_dataset = TensorDataset(X_val_normalized, y_val_tensor)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

best_val_loss = float('inf')  # Initialize with a high value
epochs_without_improvement = 0

# Training loop
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for inputs, labels in train_loader:
        # Move inputs and labels to device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)

    train_loss = running_loss / len(train_loader.dataset)
    
    # Compute validation loss
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
    
    val_loss /= len(val_loader.dataset)
    
    # Check for early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        
    # Print statistics
    print(f"Epoch [{epoch+1}/{epochs}] - Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

    if epochs_without_improvement >= patience:
        print(f"Early stopping after {epoch+1} epochs!")
        break

print('Finished Training')


Epoch [1/100] - Training Loss: 4.6184, Validation Loss: 4.5389
Epoch [2/100] - Training Loss: 4.3209, Validation Loss: 4.5837
Epoch [3/100] - Training Loss: 4.0987, Validation Loss: 4.0092
Epoch [4/100] - Training Loss: 3.8801, Validation Loss: 3.8080
Epoch [5/100] - Training Loss: 3.6757, Validation Loss: 3.6259
Epoch [6/100] - Training Loss: 3.4708, Validation Loss: 3.4570
Epoch [7/100] - Training Loss: 3.2981, Validation Loss: 3.2892
Epoch [8/100] - Training Loss: 3.1256, Validation Loss: 3.1531
Epoch [9/100] - Training Loss: 2.9619, Validation Loss: 3.0077
Epoch [10/100] - Training Loss: 2.8258, Validation Loss: 2.8685
Epoch [11/100] - Training Loss: 2.6604, Validation Loss: 2.7492
Epoch [12/100] - Training Loss: 2.5420, Validation Loss: 2.6414
Epoch [13/100] - Training Loss: 2.4189, Validation Loss: 2.5392
Epoch [14/100] - Training Loss: 2.2904, Validation Loss: 2.4254
Epoch [15/100] - Training Loss: 2.1994, Validation Loss: 2.3427
Epoch [16/100] - Training Loss: 2.0767, Validatio

In [15]:
val_loader = DataLoader(TensorDataset(X_val_normalized, y_val_tensor), batch_size=batch_size, shuffle=False)

correct_predictions = 0
total_predictions = 0

model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

accuracy = 100 * correct_predictions / total_predictions
print(f'Accuracy on the validation set: {accuracy:.2f}%')


Accuracy on the validation set: 89.90%


In [16]:
# # Save model parameters
# save_path = 'models/resnet152_100_epoch_model.pth'
# torch.save(model.state_dict(), save_path)
